### ESCOM - IIA
#### FUNDAMENTOS DE INTELIGENCIA ARTIFICIAL - PROYECTO ML
#### Semestre 2023-2 - Grupo 4BV2
--- 
##### Autores:
- **Valdés Luis Eliot Fabián**

In [142]:
# importamos la librerias necesarias
from ucimlrepo import fetch_ucirepo  # libreria de repositorios de datasets
import pandas as pd  # libreria para el manejo de dataframes

In [143]:
# cargamos el dataset
# id = 53 => el dataset de la iris flower
datasets = [53, 728, 878, 109]  # lista de datasets seleccionados
dataset = fetch_ucirepo(id=datasets[0])  # cargamos el dataset

In [144]:
# obtenemos los datos
X = dataset.data.features 
y = dataset.data.targets 

In [145]:
# creamos un dataframe con los datos
df = pd.DataFrame(X, columns=dataset.data.feature_names)
df['target'] = y

# mostramos la cantidad de registros
print(f'Cantidad de registros: {len(df)}')

# imprimimos los primeros 5 registros
print(df.head())

Cantidad de registros: 150
   sepal length  sepal width  petal length  petal width       target
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


In [146]:
# mostramos los tipos de datos de cada columna
print(f"Tipos de datos (Metodo pandas):\n{df.dtypes}\n")

Tipos de datos (Metodo pandas):
sepal length    float64
sepal width     float64
petal length    float64
petal width     float64
target           object
dtype: object



In [147]:
# mostramos los tipos de datos usando los metodos de la librearia de repositorios
dataframe_tipos = pd.DataFrame({'Variable Name': dataset.variables['name'], 'Type': dataset.variables['type']})
print(f"Tipos de datos (Metodo ucimlrepo):\n{dataframe_tipos}\n")

Tipos de datos (Metodo ucimlrepo):
  Variable Name         Type
0  sepal length   Continuous
1   sepal width   Continuous
2  petal length   Continuous
3   petal width   Continuous
4         class  Categorical



In [148]:
# por cada columna hacer lo siguiente 
# si el tipo de dato es numerico, mostrar la media, mediana, desviacion estandar, minimo y maximo
# si el tipo de dato es categorico, mostrar la cantidad de valores unicos
for col in df.columns:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        print(f"{col}:")
        print(f"\tMedia: {df[col].mean()}")
        print(f"\tMediana: {df[col].median()}")
        print(f"\tDesviacion estandar: {df[col].std()}")
        print(f"\tMinimo: {df[col].min()}")
        print(f"\tMaximo: {df[col].max()}")
    else:
        print(f"{col}:")
        print(f"\tValores unicos: {df[col].unique()}")
        print(f"\tCantidad de valores unicos: {df[col].nunique()}")

sepal length:
	Media: 5.843333333333334
	Mediana: 5.8
	Desviacion estandar: 0.828066127977863
	Minimo: 4.3
	Maximo: 7.9
sepal width:
	Media: 3.0540000000000003
	Mediana: 3.0
	Desviacion estandar: 0.4335943113621737
	Minimo: 2.0
	Maximo: 4.4
petal length:
	Media: 3.758666666666666
	Mediana: 4.35
	Desviacion estandar: 1.7644204199522626
	Minimo: 1.0
	Maximo: 6.9
petal width:
	Media: 1.1986666666666668
	Mediana: 1.3
	Desviacion estandar: 0.7631607417008411
	Minimo: 0.1
	Maximo: 2.5
target:
	Valores unicos: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
	Cantidad de valores unicos: 3


PASO 2: PREPROCESAMIENTO DE DATOS

Separamos el dataset en los vectores de entrada X & de salida Y(las clases que son definidas con tipo de dato categorico)


In [149]:
# el vector de entrada X son todas las columnas menos la columna target
vector_x = df.drop(['target'], axis=1)

# mostramos los primeros 5 registros del vector de entrada X
print(vector_x.head())

   sepal length  sepal width  petal length  petal width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2


In [150]:
# el vector de salida Y es la columna target
vector_y = df['target']

# mostramos los primeros 5 registros del vector de salida Y
print(vector_y.head())

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: target, dtype: object


In [164]:
def describe_categories(vector_x, vector_y):
    # agrupamos el vector de entrada X de acuerdo al vector de salida Y
    grouped = vector_x.groupby(vector_y)
    for name, group in grouped:
        print("\n",("="*50))
        print(f'Clase: {name}')
        print("="*50)
        # por cada grupo, accedemos a cada una de las columnas
        for col in group.columns:
            # validamos que el tipo de dato de la columna sea numerico y de ser el caso mosstramos las estadisticas
            if group[col].dtype == 'float64' or group[col].dtype == 'int64':
                print(f"{col.upper()}:")
                print(f"\t-Media: {group[col].mean()}")
                print(f"\t-Mediana: {group[col].median()}")
                print(f"\t-Desviacion estandar: {group[col].std()}")
                print(f"\t-Minimo: {group[col].min()}")
                print(f"\t-Maximo: {group[col].max()}")
            else:
                print(f"{col.upper()}:")
                print(f"\tValores unicos: {group[col].unique()}")
                print(f"\tCantidad de valores unicos: {group[col].nunique()}")
            
        

describe_categories(vector_x, vector_y)


Clase: Iris-setosa
SEPAL LENGTH:
	-Media: 5.006
	-Mediana: 5.0
	-Desviacion estandar: 0.35248968721345136
	-Minimo: 4.3
	-Maximo: 5.8
SEPAL WIDTH:
	-Media: 3.418
	-Mediana: 3.4
	-Desviacion estandar: 0.38102439795469095
	-Minimo: 2.3
	-Maximo: 4.4
PETAL LENGTH:
	-Media: 1.464
	-Mediana: 1.5
	-Desviacion estandar: 0.17351115943644546
	-Minimo: 1.0
	-Maximo: 1.9
PETAL WIDTH:
	-Media: 0.244
	-Mediana: 0.2
	-Desviacion estandar: 0.1072095030816784
	-Minimo: 0.1
	-Maximo: 0.6

Clase: Iris-versicolor
SEPAL LENGTH:
	-Media: 5.936
	-Mediana: 5.9
	-Desviacion estandar: 0.5161711470638634
	-Minimo: 4.9
	-Maximo: 7.0
SEPAL WIDTH:
	-Media: 2.7700000000000005
	-Mediana: 2.8
	-Desviacion estandar: 0.3137983233784114
	-Minimo: 2.0
	-Maximo: 3.4
PETAL LENGTH:
	-Media: 4.26
	-Mediana: 4.35
	-Desviacion estandar: 0.46991097723995795
	-Minimo: 3.0
	-Maximo: 5.1
PETAL WIDTH:
	-Media: 1.3259999999999998
	-Mediana: 1.3
	-Desviacion estandar: 0.19775268000454405
	-Minimo: 1.0
	-Maximo: 1.8

Clase: Iris-virg